In [ ]:
from typing import Callable, List
from math import sqrt
import math


# Funções básicas

## Calculo do erro

In [ ]:
def erro_absoluto(referencia: float, estimativa: float) -> float:
    return abs(referencia - estimativa)


def erro_relativo(referencia: float, estimativa: float) -> float:
    return abs((referencia, estimativa) / estimativa)


# Derivação numerica

In [ ]:
DELTA_DERIVACAO_PADRAO = 10**-4


def derivada_direita(f: Callable[[float], float], delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x)) / delta


def derivada_esquerda(f: Callable[[float], float], delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x - delta) - f(x)) / delta


def derivada_central(f: Callable[[float], float], delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x - delta)) / (2 * delta)


# Métodos iterativos

## Condições de parada

In [ ]:
MAX_ITER_PADRAO = 100


def limite_iteracao(i: int):
    return lambda est: est.i >= i


def criterio_erro_absoluto(referencia: float, erro_max: float):
    return lambda est: erro_absoluto(referencia, est.x) < erro_max


def criterio_ab(erro_max: float):
    return lambda est: est.b - est.a < 2 * erro_max


def criterio_x(erro_max: float):
    return lambda est: est.f(est.x - erro_max) * est.f(est.x + erro_max) < 0


def criterio_delta_x(delta: float):
    return lambda est: est.prior_x is not None and abs(est.x - est.prior_x) < delta


def criterio_y(dist_zero: float):
    return lambda est: abs(est.y) < dist_zero


## Método da bisseçao

In [ ]:
class EstatisticaAB:
    def __init__(
        self, f: Callable[[float], float], a: float, b: float, x: float, y: float
    ) -> None:
        self.f = f
        self.i: int = 1
        self.a = a
        self.b = b
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, a: float, b: float, x: float, y: float):
        self.i += 1
        self.a = a
        self.b = b
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, a: {self.a}, b: {self.b}, x: {self.x}, y: {self.y}"


In [ ]:
def bissecao(
    f: Callable[[float], float],
    a: float,
    b: float,
    criterios: list[Callable[[EstatisticaAB], bool]],
    observador: Callable[[EstatisticaAB], any] = lambda _: None,
    max_iter=MAX_ITER_PADRAO,
):
    if a > b:
        a, b = b, a
    i = 1
    x = (a + b) / 2
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not all(map(lambda c: c(est), criterios)) and est.i < max_iter:
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a + b) / 2
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método da falsa posição

In [ ]:
def falsa_posicao(
    f: Callable[[float], float],
    a: float,
    b: float,
    criterios: list[Callable[[EstatisticaAB], bool]],
    observador: Callable[[EstatisticaAB], any] = lambda _: None,
    max_iter=MAX_ITER_PADRAO,
):
    if a > b:
        a, b = b, a
    i = 1
    x = (a * f(b) - b * f(a)) / (f(b) - f(a))
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not all(map(lambda c: c(est), criterios)) and est.i < max_iter:
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a * f(b) - b * f(a)) / (f(b) - f(a))
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método do ponto fixo

In [ ]:
class EstatisticaX:
    def __init__(
        self,
        f: Callable[[float], float],
        fi: Callable[[float], float],
        x: float,
        y: float,
    ) -> None:
        self.i: int = 1
        self.f = f
        self.fi = fi
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, x: float, y: float):
        self.i += 1
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, x: {self.x}, y:{self.y}"


In [ ]:
def ponto_fixo(
    f: Callable[[float], float],
    fi: Callable[[float], float],
    x: float,
    criterios: list[Callable[[EstatisticaX], bool]],
    observador: Callable[[EstatisticaX], any] = lambda _: None,
    max_iter=MAX_ITER_PADRAO,
):
    i = 1
    y = f(x)
    est = EstatisticaX(f, fi, x, y)
    observador(est)
    while not all(map(lambda c: c(est), criterios)) and est.i < max_iter:
        i += 1
        x = fi(x)
        y = f(x)
        est.atualizar(x, y)
        observador(est)
    return est


## Método de Newtown-Raphson

In [ ]:
def newton_raphson(
    f: Callable[[float], float],
    x: float,
    criterios: List[Callable[[EstatisticaX], bool]],
    observador: Callable[[EstatisticaX], any] = lambda x: None,
    max_iter=MAX_ITER_PADRAO,
):
    d = derivada_central(f)
    fi = lambda x: x - f(x) / d(x)
    return ponto_fixo(f, fi, x, criterios, observador, max_iter)


# Método de Gauss-Seidel

In [ ]:
def criterio_da_distancia_relativa(e):
    def criterio(x_ant, x):
        dr = max((abs(i - j) for i, j in zip(x_ant, x)))
        return dr < e

    return criterio


def progride(a, b, x_ant):
    n = len(a)
    x = [0] * n
    for i in range(n):
        s1 = sum((a[i][j] * x_ant[j]) for j in range(i))
        s2 = sum((a[i][j] * x_ant[j] for j in range(i + 1, n)))
        x[i] = (b[i] - s1 - s2) / a[i][i]
    return x


def gauss_seidel(b, y, x_inicial, criterio, max_iter=10000):
    i = 1
    x_ant = x_inicial
    x = progride(b, y, x_ant)
    i += 1
    while not criterio(x_ant, x) and i < max_iter:
        x_ant = x
        x = progride(b, y, x_ant)
        i += 1
    return x


# Amostras

In [ ]:
def amostra_linear(x):
    return 5 * x + 2


def amostra_poli(x):
    return x**2 + x - 6


def amostra_poli_iter(x):
    return sqrt(6 - x)


# Playground 

In [ ]:
# Questão 3
b = [
    [9, -2, 1, 3, 1],
    [1, -1, 2, 6, 1],
    [2, -3, 7, 1, 0],
    [1, 6, -1, 4, 2],
    [1, -2, 2, 1, 8],
]

y = [0, 2, 1, 1, 2]

x_inicial = [1, 1, 1, 1, 1]

x = gauss_seidel(b, y, x_inicial, criterio_da_distancia_relativa(10**-4))

print(x)
